In [ ]:
! pip install kaggle-environments --upgrade -q

In [ ]:
%%writefile goose.py

import networkx as nx
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

last_action = None


def get_last_cell(last_action, current_cell, conf):
    '''Returns the player previous cell given an action and current position'''
    act_row, act_col = last_action.to_row_col()
    player_row, player_col = current_cell

    last_row = (player_row - act_row) % conf.rows
    last_col = (player_col - act_col) % conf.columns
    return (last_row, last_col)


def get_action(current_cell, next_cell, conf):
    '''Returns the action that moves the player to the specified cell given the current player position'''
    if (get_last_cell(Action.NORTH, next_cell, conf) == current_cell):
        return Action.NORTH
    elif (get_last_cell(Action.SOUTH, next_cell, conf) == current_cell):
        return Action.SOUTH
    elif (get_last_cell(Action.EAST, next_cell, conf) == current_cell):
        return Action.EAST
    elif (get_last_cell(Action.WEST, next_cell, conf) == current_cell):
        return Action.WEST
    else:
        raise Exception('No possible action found')


def game_graph(obs, conf):
    G = nx.DiGraph(nx.generators.lattice.grid_2d_graph(
        conf.rows, conf.columns, periodic=True))

    # Remove connection to previous cell
    current_cell = row_col(obs.geese[obs.index][0], conf.columns)
    if(last_action != None):
        last_cell = get_last_cell(last_action, current_cell, conf)
        G.remove_edge(current_cell, last_cell)

    for goose in obs.geese:
        if(goose != []):
            # Remove connections to body_parts except tail if it's safe.
            # If there is no food near goose head the tail will move and it's safe to keep the connections.
            goose_tail = goose[-1]
            goose_head_cell = row_col(goose[0], conf.columns)
            for body_part in goose:
                safe_tail = True
                for food in obs.food:
                    food_cell = row_col(food, conf.columns)
                    if(food_cell == goose_head_cell or food_cell in list(G.neighbors(goose_head_cell))):
                        safe_tail = False
                if(body_part != goose_tail or not safe_tail):
                    part_row, part_col = row_col(body_part, conf.columns)
                    G.remove_edges_from(list(G.in_edges((part_row, part_col))))

    return G


def agent(obs_dict, config_dict):
    '''
    This agent will find the shortest path to food from all geese in game.
    It will try to reach it only if its not the longest goose and the food 
    can be reached before the opponents.
    If we are the longest goose in game we move to the node with maximum 
    local reaching centrality (proportion of other nodes reachable from that node)
    '''
    
    global last_action

    obs = Observation(obs_dict)
    conf = Configuration(config_dict)

    G = game_graph(obs, conf)
    player_goose = obs.geese[obs.index]
    current_cell = row_col(player_goose[0], conf.columns)

    play_safe = True
    nearest_food_path = []
    for food in obs.food:
        # Find shortest path to food
        food_cell = row_col(food, conf.columns)
        try:
            food_path = nx.dijkstra_path(
                G, current_cell, food_cell)
        except nx.NetworkXNoPath:
            continue
        op_distances = []
        for goose in obs.geese:
            # Find shortest enemy path to food
            if (goose != player_goose and goose != []):
                op_current_cell = row_col(goose[0], conf.columns)
                try:
                    op_food_path = nx.dijkstra_path(
                        G, op_current_cell, food_cell)
                except nx.NetworkXNoPath:
                    continue
                op_distances.append(len(op_food_path))

        if(op_distances != [] and len(food_path) > 1):
            # Check if we can reach the food before the opponent
            if(len(food_path) < min(op_distances)):
                if(nearest_food_path == [] or len(food_path) < len(nearest_food_path)):
                    next_cell = food_path[1]
                    play_safe = False
                    nearest_food_path = food_path

    if(play_safe):
        # Go to the node that maximizes reachable nodes
        max_reaching_centrality = 0
        neighbors = list(G.neighbors(current_cell))
        if(len(neighbors) > 0):
            for neighbor in neighbors:
                reaching_centrality = nx.local_reaching_centrality(G, neighbor)
                if(reaching_centrality >= max_reaching_centrality):
                    max_reaching_centrality = reaching_centrality
                    next_cell = neighbor
        else:
            # We lost the game, return random action
            return Action.NORTH.name

    last_action = get_action(current_cell, next_cell, conf)
    return get_action(current_cell, next_cell, conf).name

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("hungry_geese")
env.run(["goose.py", "greedy", "greedy", "greedy"])

In [ ]:
env.render(mode="ipython", width=800, height=600)